In [106]:
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [107]:
df1 = pd.read_csv("train.csv")
df2 = pd.read_csv("test.csv")

In [108]:
df1 = df1.drop(['PID','Street', 'Utilities',  'Condition_2', 'Roof_Matl', 'Heating', 'Pool_QC', 'Misc_Feature', 'Low_Qual_Fin_SF', 'Pool_Area', 'Longitude','Latitude'], axis=1)
df2 = df2.drop(['PID','Street', 'Utilities',  'Condition_2', 'Roof_Matl', 'Heating', 'Pool_QC', 'Misc_Feature', 'Low_Qual_Fin_SF', 'Pool_Area', 'Longitude','Latitude'], axis=1)
df2

,MS_SubClass,MS_Zoning,Lot_Frontage,Lot_Area,Alley,Lot_Shape,Land_Contour,Lot_Config,Land_Slope,Neighborhood,...,Open_Porch_SF,Enclosed_Porch,Three_season_porch,Screen_Porch,Fence,Misc_Val,Mo_Sold,Year_Sold,Sale_Type,Sale_Condition
0,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,0,8926,No_Alley_Access,Slightly_Irregular,Lvl,Corner,Gtl,Edwards,...,0,0,0,160,Minimum_Privacy,0,10,2009,COD,Abnorml
1,Two_Story_1945_and_Older,Residential_Medium_Density,57,6876,No_Alley_Access,Regular,Lvl,Inside,Gtl,Iowa_DOT_and_Rail_Road,...,0,213,0,0,Minimum_Privacy,0,8,2009,WD,Normal
2,Two_Story_1946_and_Newer,Residential_Low_Density,0,13869,No_Alley_Access,Moderately_Irregular,Lvl,Corner,Gtl,Gilbert,...,75,0,0,0,No_Fence,0,7,2007,WD,Normal
3,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,87,10000,No_Alley_Access,Slightly_Irregular,Lvl,Corner,Gtl,North_Ames,...,0,0,0,385,No_Fence,0,2,2010,WD,Normal
4,One_and_Half_Story_Finished_All_Ages,Residential_Medium_Density,60,10320,Gravel,Regular,Lvl,Inside,Gtl,Iowa_DOT_and_Rail_Road,...,0,239,0,0,Minimum_Privacy,0,6,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
874,Two_Story_1946_and_Newer,Residential_Low_Density,80,11584,No_Alley_Access,Regular,Lvl,Inside,Gtl,Northwest_Ames,...,88,216,0,0,No_Fence,0,11,2007,WD,Normal
875,Two_Story_1946_and_Newer,Residential_Low_Density,72,7200,No_Alley_Access,Regular,Lvl,Inside,Gtl,Edwards,...,0,0,0,0,Minimum_Privacy,0,2,2008,WD,Normal
876,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,84,11670,No_Alley_Access,Slightly_Irregular,Lvl,Corner,Gtl,Somerset,...,191,0,0,0,No_Fence,0,3,2007,WD,Normal
877,Two_Story_PUD_1946_and_Newer,Floating_Village_Residential,30,3215,Paved,Regular,Lvl,Inside,Gtl,Somerset,...,172,0,0,0,No_Fence,0,4,2010,ConLD,Normal


In [109]:
winsor = ["Lot_Frontage", "Lot_Area", "Mas_Vnr_Area", "BsmtFin_SF_2", "Bsmt_Unf_SF", "Total_Bsmt_SF", "Second_Flr_SF", 'First_Flr_SF', "Gr_Liv_Area", "Garage_Area", "Wood_Deck_SF", "Open_Porch_SF", "Enclosed_Porch", "Three_season_porch", "Screen_Porch", "Misc_Val"]
for col in winsor:
    df1[col] = df1[col].clip(lower=0, upper=df1[col].quantile(0.95))
    df2[col] = df2[col].clip(lower=0, upper=df2[col].quantile(0.95))

In [110]:
y_train = np.log(df1[['Sale_Price']].values)
df1 = df1.drop(['Sale_Price'], axis=1)

In [111]:
df1.Garage_Yr_Blt.fillna(df1.Year_Built, inplace=True)
df2.Garage_Yr_Blt.fillna(df2.Year_Built, inplace=True)

In [112]:
cols=[]
columns = df1.columns.to_list()
categorical_feature_mask = df1.dtypes==object
categorical_cols = df1.columns[categorical_feature_mask].tolist()
for i,c in enumerate(columns):
    if c in categorical_cols:
        cols.append(i)

In [113]:
enc = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(categories='auto', handle_unknown='ignore'), cols)],   # The column numbers to be transformed (here is [0] but can be [0, 1, 3])
    remainder='passthrough'                                         # Leave the rest of the columns untouched
)
enc.fit(df1)
X_train = enc.fit_transform(df1).toarray()
X_test = enc.transform(df2).toarray()

In [114]:
from sklearn.linear_model import Ridge
clf = Ridge(alpha=1)
clf.fit(X_train, y_train)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=None, solver='auto', tol=0.001)

In [115]:
y_test = np.exp(clf.predict(X_test))

In [116]:
df3 = pd.read_csv("test_y.csv")

In [117]:
y_real = df3[['Sale_Price']].values

In [118]:
math.sqrt( (1/y_real.shape[0]) * np.sum((np.log(y_test) - np.log(y_real))**2))

0.12193873471131782